# Asignación 1
En esta asignación reduciremos grandes datasets usando la lógica map-reduce.

leed el directorio `activations_xml` en vuestro ordenador.

Cada fichero XML contiene los datos de todos los dispositivos activados por los clientes durante 3 años. Ejemplo: 
```xml
<activations>
	<activation timestamp="2013-1-18 11:53.000" type="phone">
		<account-number>454721</account-number>
		<device-id>d051735e-cd2d-11e8-a49d-b86b239563ce</device-id>
		<phone-number>66641135</phone-number>
		<model>WIKO</model>
	</activation>
	<activation timestamp="2013-1-21 03:55.000" type="phone">
		<account-number>239096</account-number>
		<device-id>d051735f-cd2d-11e8-a2c0-b86b239563ce</device-id>
		<phone-number>42743767</phone-number>
		<model>HTC</model>
	</activation>
    ...............
</activations>
```
Por conveniencia se suministran las funciones que parsean el formato XML. Estas funciones son:
```python
import xml.etree.ElementTree as ElementTree

# Dada una cadena str que contiene XML parsea el string y retorna un iterador con los registros XML de activaciones 
# individuales (Elementos) que contiene el string
def getActivations(s):
    filetree = ElementTree.fromstring(s)
    return filetree.getiterator('activation')
    
# Dado un registro de activacion (XML Elemento), devuelve el nombre del modelo
def getModel(activation):
    return activation.find('model').text 

# Dado un registro de activacion (XML Elemento), devuelve el numero de cliente
def getAccount(activation):
    return activation.find('account-number').text 
```
Para procesar los ficheros, se debe hacer:
1. Usar wholeTextFiles para crear un RDD de activaciones. El RDD resultante consistirá de tuplas donde la clave es el nombre del fichero y el valor será el contenido del fichero (XML) como un string de caracteres.
2. Cada fichero XML puede contener muchos registros de activación. Usar flatMap para mapear el contenido de cada fichero a una colección de registros XML, llamando a la función getActivations. getActivations coge un string XML y devuelve una colección de registros XML y flatMap mapea cada registro a un elemento del RDD.
3. Mapear cada registro de activación a un string en formato account-number:model, utilizando las funciones getAccount y getModel para encontrar los valores.
4. Guardar el RDD resultante a un fichero.
5. Mapear cada activación para crear un nuevo `pairRDD` con formato (account-number,model) sobre cuya información contestaremos las preguntas de negocio que se realizan, usando las mismas funciones getAccount y getModel.

PREGUNTAS:
1. Mostrar las 5 top accounts con el mayor número de activaciones. Hint: construir el `pairRDD` y usar `reduceByKey()` y `sortBy()`.
2. Mostrar los 5 top modelos con el mayor número de activaciones. Hint: construir el `pairRDD` y usar `reduceByKey()` y `sortBy()`.
3. Mostrar cuántos clientes distintos han activado un modelo. Hint: construir el `pairRDD` y usar `distinct()`
4. Mostrar cuántos clientes han activado un modelo para cada frecuencia. Hint: intentar dar la vuelta a la clave-valor, pasando de (account-number,activaciones) a (activaciones,account-number) y usar `countByKey()`.
5. Mostrar los modelos de las 5 cuentas que más activaciones tienen. Hint: De la respuesta a la pregunta 1 sabemos las cuentas que más activaciones tienen. Ahora ddebemos sacar cuáles han sido los modelos que han activado. construir el `pairRDD` y usar `groupByKey()` y `sortBy()`.

In [1]:
import os
import sys

os.environ['SPARK_HOME'] = "C:\\spark-3.0.3-bin-hadoop2.7\\"
SPARK_HOME = os.environ['SPARK_HOME']

# Añadimos los correspondientes paths de las librerias de python
sys.path.insert(0, os.path.join(SPARK_HOME, "python"))
sys.path.insert(0, os.path.join(SPARK_HOME, "python", "lib"))
sys.path.insert(0, os.path.join(SPARK_HOME, "python", "lib","pyspark.zip"))
sys.path.insert(0, os.path.join(SPARK_HOME, "python", "lib", "py4j-0.10.9-src.zip"))

# Importamos la funcion
from pyspark.sql import SparkSession

# Creamos la sesion
spark = SparkSession \
            .builder \
            .master("local[*]") \
            .appName("Lab4") \
            .config("spark.executor.memory", "6g") \
            .config("spark.cores.max", "4") \
            .getOrCreate()


# Creamos el sparkContext de la sesion    
sc = spark.sparkContext

In [2]:
import xml.etree.ElementTree as ElementTree

# Given a string containing XML, parse the string, and 
# return an iterator of activation XML records (Elements) contained in the string
def getActivations(s):
    filetree = ElementTree.fromstring(s)
    return filetree.getiterator('activation')

# Given an activation record (XML Element), return the model name
def getModel(activation):
    return activation.find('model').text 

# Given an activation record (XML Element), return the account number 
def getAccount(activation):
    return activation.find('account-number').text 

### 1. Usar `wholeTextFiles()` con el directorio xml

In [3]:


xmlRDD = sc.wholeTextFiles("./data/activations-xml/*.xml",minPartitions=None, use_unicode=True)
# print(jsonRDD)
xmlRDD.take(1)

[('file:/C:/Users/danny/ZZ05 LABS PYSPARK/data/activations-xml/2013-01.xml',
  '<activations>\r\n\t<activation timestamp="2013-1-24 18:39.000" type="phone">\r\n\t\t<account-number>6845</account-number>\r\n\t\t<device-id>8126d422-f3c7-11e8-897c-b86b239563ce</device-id>\r\n\t\t<phone-number>6610058</phone-number>\r\n\t\t<model>PLUM</model>\r\n\t</activation>\r\n\t<activation timestamp="2013-1-11 19:38.000" type="phone">\r\n\t\t<account-number>361</account-number>\r\n\t\t<device-id>8127220c-f3c7-11e8-8a29-b86b239563ce</device-id>\r\n\t\t<phone-number>6240295</phone-number>\r\n\t\t<model>XIAOMI</model>\r\n\t</activation>\r\n\t<activation timestamp="2013-1-09 05:47.000" type="phone">\r\n\t\t<account-number>8724</account-number>\r\n\t\t<device-id>8127220d-f3c7-11e8-9e2a-b86b239563ce</device-id>\r\n\t\t<phone-number>6189944</phone-number>\r\n\t\t<model>LEECO</model>\r\n\t</activation>\r\n\t<activation timestamp="2013-1-04 18:58.000" type="phone">\r\n\t\t<account-number>5840</account-number>\r

### 2. Usar `flatMap()` para leer cada activación en un registro del RDD

In [4]:
xmlRDD1 = xmlRDD.map(lambda x: x[1])\
            .flatMap(lambda x: getActivations(x))

xmlRDD1.take(10)

[<Element 'activation' at 0x00000286B0291CC8>,
 <Element 'activation' at 0x00000286B02984F8>,
 <Element 'activation' at 0x00000286B0298868>,
 <Element 'activation' at 0x00000286B0298D68>,
 <Element 'activation' at 0x00000286B0298F48>,
 <Element 'activation' at 0x00000286B029C4A8>,
 <Element 'activation' at 0x00000286B029C9A8>,
 <Element 'activation' at 0x00000286B0285CC8>,
 <Element 'activation' at 0x00000286B0291E58>,
 <Element 'activation' at 0x00000286B02916D8>]

### 3. Mapear cada activación a un registro "account-number:model-name"

In [5]:
xmlRDD2 = xmlRDD\
            .map(lambda x: x[1])\
            .flatMap(lambda x: getActivations(x))\
            .map(lambda b: getAccount(b)+":"+getModel(b))

xmlRDD2.take(1)

['6845:PLUM']

### 4. Guardar el RDD en un fichero

In [7]:
xmlRDD2.saveAsTextFile("./data/practicaSpark")

Py4JJavaError: An error occurred while calling o93.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/C:/Users/danny/ZZ05 LABS PYSPARK/data/practicaSpark already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:294)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1090)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:388)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1088)
	at org.apach

### 5. Nuevo mapeo `map` para resolver las preguntas

#### 5.1 Mostrar las top 5 cuentas con el mayor número de activaciones

In [8]:
xmlRDD3 = xmlRDD\
            .map(lambda x: x[1])\
            .flatMap(lambda x: getActivations(x))\
            .map(lambda b: (getAccount(b),1))\
            .reduceByKey(lambda x,y: x+y)\
            .sortBy(lambda x: x[1], False)
                
xmlRDD3.take(5)


[('3829', 12), ('2219', 12), ('4084', 12), ('5158', 11), ('3007', 11)]

#### 5.2 Mostrar los top 5 modelos que más han activado los clientes

In [9]:
xmlRDD4 = xmlRDD\
            .map(lambda x: x[1])\
            .flatMap(lambda x: getActivations(x))\
            .map(lambda x: (getModel(x),1))\
            .reduceByKey(lambda x,y: x+y)\
            .sortBy(lambda x: x[1], False)

                
xmlRDD4.take(5)

[('HP', 1073), ('LEECO', 1064), ('LAVA', 1060), ('APPLE', 1058), ('HTC', 1042)]

#### 5.3 Mostrar cuántas cuentas distintas han activado algún modelo

In [10]:
xmlRDD5 = xmlRDD\
            .map(lambda x: x[1])\
            .flatMap(lambda x: getActivations(x))\
            .map(lambda x: getAccount(x))\
            .distinct()
xmlRDD5.count()

9731

#### 5.4 Mostrar cuántas cuentas han activado un modelo para cada frecuencia

In [11]:
xmlRDD6 = xmlRDD\
            .map(lambda x: x[1])\
            .flatMap(lambda x: getActivations(x))\
            .map(lambda x: (getAccount(x),1))\
            .reduceByKey(lambda x,y:x+y)\
            .map(lambda x: (x[1],x[0]))\
            .countByKey()
xmlRDD6

defaultdict(int,
            {4: 1925,
             7: 428,
             2: 1759,
             6: 798,
             9: 75,
             5: 1339,
             3: 2160,
             1: 989,
             12: 3,
             8: 219,
             10: 25,
             11: 11})

#### 5.5 Mostrar los modelos de las 5 cuentas que más activaciones han realizado

In [23]:
xmlRDD7 = xmlRDD\
            .map(lambda x: x[1])\
            .flatMap(lambda x: getActivations(x))\
            .map(lambda x: (getAccount(x),1))\
            .reduceByKey(lambda x,y:x+y)\
            .sortBy(lambda x: x[1], False)\
            .map(lambda x: x[0])

top5=xmlRDD7.take(5)
top5

['3829', '2219', '4084', '5158', '3007']

In [25]:
xmlRDD8 = xmlRDD\
            .map(lambda x: x[1])\
            .flatMap(lambda x: getActivations(x))\
            .map(lambda x: (getAccount(x),getModel(x)))\
            .groupByKey()
nombre=xmlRDD8.collect()
nombre

[('6845', <pyspark.resultiterable.ResultIterable at 0x286b4d6bd88>),
 ('361', <pyspark.resultiterable.ResultIterable at 0x286b4d6b588>),
 ('2121', <pyspark.resultiterable.ResultIterable at 0x286b4f15d08>),
 ('901', <pyspark.resultiterable.ResultIterable at 0x286b4333b88>),
 ('3850', <pyspark.resultiterable.ResultIterable at 0x286b4f15348>),
 ('3223', <pyspark.resultiterable.ResultIterable at 0x286b4f3f4c8>),
 ('6322', <pyspark.resultiterable.ResultIterable at 0x286b4f3f388>),
 ('8698', <pyspark.resultiterable.ResultIterable at 0x286b4f15648>),
 ('8650', <pyspark.resultiterable.ResultIterable at 0x286b4d82e08>),
 ('2287', <pyspark.resultiterable.ResultIterable at 0x286b4f29b48>),
 ('2550', <pyspark.resultiterable.ResultIterable at 0x286b4f29d48>),
 ('8262', <pyspark.resultiterable.ResultIterable at 0x286b4f29ec8>),
 ('7055', <pyspark.resultiterable.ResultIterable at 0x286b4f29648>),
 ('3829', <pyspark.resultiterable.ResultIterable at 0x286b4f291c8>),
 ('2753', <pyspark.resultiterable.Re

In [26]:
for i in nombre:
    if i[0] in top5:
        print("\n" + i[0])
        for j in i[1]:
            print(j,end=" ")


3829
MEIZU WIKO MOTOROLA ALCATEL PANASONIC ZTE VODAFONE MOTOROLA ONEPLUS XIAOMI YU ONEPLUS 
5158
PANASONIC ACER MICROSOFT VIVO ENERGIZER NOKIA HUAWEI BLU MICROMAX VIVO ALCATEL 
3007
MEIZU LAVA ONEPLUS SAMSUNG VODAFONE ENERGIZER ALCATEL ONEPLUS XIAOMI HUAWEI SAMSUNG 
2219
LAVA XIAOMI VIVO APPLE MAXWEST TOSHIBA PLUM PLUM ALCATEL LG YU ACER 
4084
ZTE ONEPLUS OPPO YU LENOVO HTC BLU PANASONIC MOTOROLA HTC ZTE ALCATEL 

In [ ]:
xmlRDD9=xmlRDD7\
    .join(xmlRDD8)\
    .flatMap(lambda x:x[2])

xmlRDD9.take(5)

#### EXTRA a - Cuántas cuentas distintas activaron cada modelo

In [56]:
xmlRDD10 = xmlRDD\
            .map(lambda x: x[1])\
            .flatMap(lambda x: getActivations(x))\
            .map(lambda x: ((getModel(x)), getAccount(x)))\
            .distinct()\
            .countByKey()

xmlRDD10

defaultdict(int,
            {'PLUM': 959,
             'ACER': 902,
             'LENOVO': 961,
             'LEECO': 1011,
             'VIVO': 948,
             'GOOGLE': 981,
             'ENERGIZER': 953,
             'XIAOMI': 896,
             'TOSHIBA': 958,
             'MICROSOFT': 915,
             'LAVA': 1009,
             'MICROMAX': 950,
             'HP': 1015,
             'ONEPLUS': 922,
             'CAT': 936,
             'MOTOROLA': 927,
             'SAMSUNG': 968,
             'ALCATEL': 920,
             'VODAFONE': 925,
             'YU': 975,
             'OPPO': 965,
             'HUAWEI': 922,
             'MAXWEST': 903,
             'SONY': 979,
             'WIKO': 937,
             'VERYKOOL': 923,
             'BLU': 965,
             'PANASONIC': 931,
             'APPLE': 1002,
             'MEIZU': 965,
             'NOKIA': 961,
             'BLACKBERRY': 963,
             'ZTE': 939,
             'HTC': 987,
             'ASUS': 953,
             

#### EXTRA b - Cuántas cuentas han activado cada modelo para cada frecuencia

In [59]:
xmlRDD11 = xmlRDD\
            .map(lambda x: x[1])\
            .flatMap(lambda x: getActivations(x))\
            .map(lambda x: ((getModel(x),getAccount(x)),1))\
            .reduceByKey(lambda x,y: x+y)\
            .map(lambda x: ((x[1],x[0][0]),x[0][1]))\
            .countByKey()


xmlRDD11

defaultdict(int,
            {(1, 'PLUM'): 904,
             (1, 'ACER'): 865,
             (1, 'LENOVO'): 913,
             (1, 'LEECO'): 961,
             (1, 'VIVO'): 904,
             (1, 'GOOGLE'): 932,
             (1, 'ENERGIZER'): 914,
             (1, 'XIAOMI'): 850,
             (1, 'TOSHIBA'): 914,
             (1, 'MICROSOFT'): 870,
             (1, 'LAVA'): 960,
             (1, 'MICROMAX'): 897,
             (2, 'HP'): 54,
             (1, 'ONEPLUS'): 877,
             (1, 'CAT'): 879,
             (1, 'MOTOROLA'): 873,
             (1, 'SAMSUNG'): 934,
             (1, 'ALCATEL'): 876,
             (1, 'VODAFONE'): 882,
             (1, 'YU'): 925,
             (1, 'OPPO'): 926,
             (1, 'HUAWEI'): 883,
             (2, 'VIVO'): 41,
             (2, 'GOOGLE'): 47,
             (1, 'HP'): 959,
             (1, 'MAXWEST'): 853,
             (1, 'SONY'): 940,
             (2, 'WIKO'): 43,
             (1, 'VERYKOOL'): 879,
             (1, 'WIKO'): 893,
            